In [68]:
import scipy.io
import mne
from scipy.signal import butter, filtfilt
import os
import numpy as np
from scipy import stats

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

from matplotlib import pyplot as plt
mne.set_log_level('error')


In [2]:
data_root = 'C:/Data/UHD_EEG/'
subjects = ['S1', 'S2', 'S3', 'S4', 'S5']
dominant_hand = ['left','right','right','right','right']

mapping = {0: "No instruction", 1: "Rest", 2: "thumb", 3: "index", 4: "middle", 5: "ring", 6: "little"}

not_ROI_channels = ['c255', 'c256', 'c254', 'c251', 'c239', 'c240', 'c238', 'c235', 'c224', 'c222', 'c223', 'c219', 'c220', 'c221', 'c215', 'c216', 'c217', 'c213', 'c212', 'c211', 'c210', 'c209', 'c112', 'c110', 'c107', 'c108', 'c103', 'c104', 'c105', 'c101', 'c100', 'c99', 'c98', 'c97', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c14', 'c15', 'c16', 'c23', 'c29', 'c26', 'c17', 'c18', 'c20', 'c19', 'c21', 'c24', 'c22', 'c25', 'c28', 'c33', 'c35', 'c38', 'c42', 'c81', 'c34', 'c37', 'c41', 'c45', 'c36', 'c40', 'c44', 'c39', 'c43', 'c145', 'c147', 'c150', 'c154', 'c157', 'c153', 'c149', 'c146', 'c93', 'c159', 'c156', 'c152', 'c148', 'c95', 'c160', 'c158', 'c155', 'c151', 'c96', 'c202', 'c198', 'c195', 'c193']

# Subject 1 bad channels:
bad_channels =['c65', 'c66', 'c67', 'c68', 'c69', 'c70', 'c71', 'c72', 'c73', 'c74', 'c75', 'c76', 'c77', 'c78', 'c79', 'c80']

In [3]:
def get_montage(hemishpere):
    mat = scipy.io.loadmat(os.path.join(data_root, 'montage', f'montage_256_{hemishpere}_hemisphere.mat'))
    return mat['pos_256']

left_handed_montage = get_montage('right')
right_handed_montage = get_montage('left')

In [4]:
def load_run(subject_id, run, describe = True):
    # LOAD DATA
    #run = subject + '_run' + str(run) + '.mat'
    subject = subjects[subject_id]
    mat = scipy.io.loadmat(os.path.join(data_root, 'rawdata', subject, run))
    data = mat['y']
    data = data[1:] # remove timestamp
    ch_names = ['c' + str(i) for i in range(1, 257)]
    ch_names.append('STIM')
    info = mne.create_info(ch_names = ch_names, sfreq=mat['SR\x00'][0][0]) #, ch_types='eeg')
    raw = mne.io.RawArray(data, info)

    # SET CHANNEL TYPES
    for ch in info.ch_names:
        if ch != 'STIM':
            raw.set_channel_types({ch: 'eeg'})
        else:
            raw.set_channel_types({ch: 'stim'})

    events = mne.find_events(raw, stim_channel='STIM')
    annot_from_events = mne.annotations_from_events(events, event_desc=mapping, sfreq=raw.info['sfreq'])
    raw.set_annotations(annot_from_events)
    raw.drop_channels(['STIM'])

    # SET MONTAGE
    if dominant_hand[subject_id] == 'left':
        montage = left_handed_montage
    else:
        montage = right_handed_montage

    montage = mne.channels.make_dig_montage(ch_pos=dict(zip(ch_names, montage)), coord_frame='head')
    raw.set_montage(montage)

    if describe:
        raw.describe()
    return raw

In [5]:
def load_subject(subject_id, describe = True):
    runs = []
    run_files = os.listdir(os.path.join(data_root, 'rawdata', subjects[subject_id]))
    for file in run_files:
        runs.append(load_run(subject_id, file, describe))
    return runs

In [8]:
raw_runs = load_subject(subject_id = 0, describe = False)

<RawArray | 256 x 135559 (225.9 s), ~265.0 MB, data loaded>
 ch  name  type  unit        min         Q1     median         Q3        max
  0  c1    EEG   µV    -85493210937.50  -35441191406.25  1677024414.06  36613792968.75  81199593750.00
  1  c2    EEG   µV    -85721015625.00  -35531160156.25  1664257812.50  36644519531.25  81226898437.50
  2  c3    EEG   µV    -85876343750.00  -35615492187.50  1649694213.87  36730896484.38  81483765625.00
  3  c4    EEG   µV    -85769679687.50  -35548326171.88  1663018066.41  36666996093.75  81330179687.50
  4  c5    EEG   µV    -85758578125.00  -35550046875.00  1659248779.30  36668800781.25  81367742187.50
  5  c6    EEG   µV    -85414617187.50  -35385037109.38  1647537475.59  36503046875.00  80972781250.00
  6  c7    EEG   µV    -85671015625.00  -35482373046.88  1650583496.09  36601558593.75  81176257812.50
  7  c8    EEG   µV    -85453140625.00  -35408878906.25  1628080566.41  36478734375.00  80898117187.50
  8  c9    EEG   µV    -85142101562.50 

In [75]:
def preprocess(orig):
    run = orig.copy()

    # run.drop_channels(bad_channels)
    # run.drop_channels(not_ROI_channels)

    #run.notch_filter(60)

    run.filter(8, 25, fir_design='firwin')

    run.set_eeg_reference('average', projection=True)

    #run.apply_proj()

    # baseline = run._data[:, :-int(25*run.info['sfreq'])]
    # run._data = run._data - baseline.mean(axis=1, keepdims=True)

    run = run.resample(5)
    
    return run

In [76]:
preprocessed_runs = []
for run in raw_runs:
    preprocessed_runs.append(preprocess(run))

In [77]:
preprocessed_runs[0]._data.shape

(256, 1130)

In [78]:
epochs = []
for run in preprocessed_runs:
    events, event_ids = mne.events_from_annotations(run)
    asd = mne.Epochs(run, events, baseline = (-1, -0.5), event_id= event_ids, tmin=-1, tmax=7, preload=True)
    epochs.append(asd)
epochs = mne.concatenate_epochs(epochs)
epochs = epochs.crop(tmin=-0.5, tmax=7)

In [79]:
epochs

Number of events,260
Events,Rest: 10index: 50little: 50middle: 50ring: 50thumb: 50
Time range,-0.400 – 7.000 sec
Baseline,-1.000 – -0.500 sec


In [80]:
epochs[0].get_data().shape

(1, 256, 38)

In [81]:
X = epochs.get_data()
y = epochs.events[:, -1]

X = X.reshape(X.shape[0], -1) # reshape to SVM

print(X.shape)
print(y.shape)

(260, 9728)
(260,)


In [82]:
# thumb, index, middle, ring, and little finger respectively.

thumb_indices = np.where(y == 2)
index_indices = np.where(y == 3)
middle_indices = np.where(y == 4)
ring_indices = np.where(y == 5)
little_indices = np.where(y == 6)

thumb = X[thumb_indices]
index = X[index_indices]
middle = X[middle_indices]
ring = X[ring_indices]
little = X[little_indices]

In [83]:
middle_vs_ring = np.concatenate((middle, ring), axis=0)
y_middle_vs_ring = np.concatenate((np.zeros(middle.shape[0]), np.ones(ring.shape[0])), axis=0)
X = middle_vs_ring
y = y_middle_vs_ring


In [84]:
# Grid search for best parameters


# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf','linear'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4],
                        'C': [0.001, 0.01, 0.1, 1, 10]}]
grid = GridSearchCV(SVC(), tuned_parameters, cv=StratifiedKFold(n_splits=5), scoring='accuracy')
grid.fit(X, y)
print(grid.best_params_)
print(grid.best_score_)



{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
0.6199999999999999


In [85]:
# K fold cross validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svc = SVC()
    svc.fit(X_train, y_train)
    print(svc.score(X_test, y_test))
    accuracy_scores.append(svc.score(X_test, y_test))
avg_accuracy = np.mean(accuracy_scores)
print("Average accuracy: ", avg_accuracy)

0.6
0.8
0.7
0.6
0.6
0.7
0.5
0.6
0.7
0.2
Average accuracy:  0.6
